# qBraid Open Challenge

In this open challenge, submissions will be judged on the most creative/compelling use of the [qBraid-SDK](https://qbraid-qbraid.readthedocs-hosted.com/en/latest/sdk/overview.html). The only additional requirement is that you must in some way use [qBraid quantum jobs](https://github.com/qBraid/qbraid-lab-demo/blob/main/qbraid_quantum_jobs.ipynb) to interface with [Amazon Braket supported devices](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html).

#### Resources

- [qBraid demo notebooks](https://github.com/qBraid/qbraid-lab-demo)
- [qBraid SDK: user guide](https://qbraid-qbraid.readthedocs-hosted.com/en/latest/sdk/overview.html)
- [qBraid SDK: API reference](https://qbraid-qbraid.readthedocs-hosted.com/en/latest/api/qbraid.html)
- [qBraid CLI: quantum jobs commands](https://qbraid-qbraid.readthedocs-hosted.com/en/latest/cli/jobs.html)

In [1]:
!qbraid jobs enable haqs

You have already enabled qBraid Quantum Jobs in the haqs environment.


In [2]:
TEAM_NAME = "Kossakowski"  # enter team name

In [3]:
import qbraid

from qbraid import (
    get_devices,
    get_jobs,
    circuit_wrapper,
    device_wrapper,
    job_wrapper,
    SUPPORTED_PROGRAM_TYPES,
)
from qbraid.api import ibmq_least_busy_qpu, verify_config
from qbraid.interface import random_circuit, to_unitary, circuits_allclose, draw

In [4]:
qbraid.__version__

'0.1.4'

In [6]:
# Your code!
#!pip install dimod
#!pip install dwave-system
#!pip install amazon-braket-sdk --user
#!pip install amazon-braket-ocean-plugin

In [7]:
from qbraid import get_devices, get_jobs, device_wrapper, job_wrapper
from qbraid.interface import random_circuit
from pprint import pprint as pp
import dimod
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dwave.system.composites import EmbeddingComposite

In [8]:
devices = get_devices(filters={
    'vendor': 'AWS',
    'provider': 'D-Wave',
    'paradigm': 'quantum-annealer',
    'status': 'ONLINE'
})

In [9]:
# The code in next two cells comes from qbraid-tutorials/amazon_braket_examples/quantum_annealing/Dwave_Anatomy.ipynb

# simple QUBO preparation
quadratic = {(0, 1): 1.0, (0,4): 2.0, (1, 2): 1.0, (1,4): 1.0, (2, 3): 1.0, (3, 4): 1.0}
linear = {0: +1.0}
offset = 0.0

# set parameters
num_reads = 10
vartype = dimod.SPIN

bqm = dimod.BinaryQuadraticModel(linear, quadratic, offset, vartype)

In [10]:
# run classical simulated annealing
sa_sampler = dimod.SimulatedAnnealingSampler()
sa_response = sa_sampler.sample(bqm, num_reads=num_reads)

print(sa_response.aggregate())
min_energy = min(sa_response.data_vectors['energy'])
print('Minimal energy:', min_energy)

   0  1  2  3  4 energy num_oc.
0 -1 -1 +1 -1 +1   -6.0      10
['SPIN', 1 rows, 10 samples, 5 variables]
Minimal energy: -6.0


In [ ]:
# run on actual D-Wave quantum annealer
qbraid_id = "aws_dwave_advantage_system4"
quantum_annealer = device_wrapper(qbraid_id)
pp(quantum_annealer.info)
pp(quantum_annealer)

In [ ]:
print(type(quantum_annealer.vendor_dlo))
print(quantum_annealer._arn)

The functionality below (commented code) is not working yet, hence a workaround for accessing quantum annealer via qBraid SDK is needed:

In [ ]:
#qjob = quantum_annealer.run(bqm)

In [ ]:
# workaround - uploading task via Amazon Braket API:
my_bucket = f"amazon-braket-My-Test-Bucket-Name"
my_prefix = "My-Test-Folder-Name"
s3_folder = (my_bucket, my_prefix)

sampler = BraketDWaveSampler(s3_folder,device_arn=quantum_annealer._arn)
sampler = EmbeddingComposite(sampler)

In [ ]:
print(sampler)
print(bqm)

In [ ]:
qa_response = sampler.sample(bqm, num_reads=10)
print(qa_response.info["taskMetadata"]["id"])

In [ ]:
get_jobs(filters={"qbraidDeviceId": qbraid_id})

In [ ]:
saved_job_id = 'aws_dwave_advantage_system4-kmnalecz-qjob-su0nt0tt640yli2du4dp'
qjob = job_wrapper(saved_job_id)
qjob.wait_for_final_state()
status = qjob.status()
qresult = qjob.result()

In [ ]:
print(status)
print(type(qresult))
print(qresult.__dict__)